In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.resnet50 import ResNet50

import numpy as np
import random
import math
import csv
import cv2
import os

"""
Bu dosyada Xception hazır olan model kullanılacak.
"""

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import MaxPool2D # üstteki MaxPooling2D ile aynı şey. ister onu ister bunu kullan.
#kodda ikisi de kullanıldığı için eklendi. yoksa biri ile yapılsaydı da olurdu.
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers import SGD

In [ ]:
inputBasePath    = r"/content/Project/Dataset"
outputBasePath =  r"/content/Project/DatasetArray" #bu klasörlerin daha önce oluşturulmuş olması gerek
image_width = 299
image_height = 299
classes = ['2', '3', '4', '5']

In [ ]:
"""
Bu kodda veri setinden ilgili katergorileri bütün fotoğraflarını okunup numpy dizisine atanıyor ve bir dosyda kaydediliyor. Sonradan farklı model
  denenmek istediğinde fotoğrafları baştan okumamamk için. iki tane npy dosyası oluşturuluyor biri "images" girdi olarak çıktı olarak da ilgili
  fotoğrafın ait olduğu kategori o da "labels" dosyasında kaydediliyor.

"""
os.chdir(inputBasePath)

X = []
Y = []

for class1 in classes:
    os.chdir(class1)
    print('=> ' + class1)

    for file in os.listdir('./'):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img = cv2.imread(file)
        if img is None:
            print(f"{file} adlı resim okunamadı.")
            continue

        img = cv2.resize(img, (image_width, image_height))
        img = vgg16.preprocess_input(img)
        X.append(img)
        Y.append(class1)

    os.chdir('..')

X = np.array(X).reshape(-1, image_width, image_height, 3)
Y = np.array(Y)

os.chdir('..')
os.makedirs("DatasetArray", exist_ok=True)
os.chdir("DatasetArray")

np.save(f"{image_width}x{image_height}_images", X)
np.save(f"{image_width}x{image_height}_labels", Y)

print("[BİLGİ - AŞAMA 1] NUMPY DİZİSİ OLUŞTURMA TAMAMLANDI\n")

In [ ]:
# npy diye kaydedilen x ve y değerleri okunup ilki data ikincisi labels değişkenine atanıyor.
data = np.load(r"" + outputBasePath +f"/{image_width}x{image_height}_images.npy")
labels = np.load(r"" + outputBasePath +f"/{image_width}x{image_height}_labels.npy")
data.shape

In [ ]:
# sınıflarımız "2,3,4,5" LabelEncode sınıfını kullanarak çıktı katmananında her birinin hangi neorunun bir geldiğinde ele alınacağını hesaplıyoruz
labelEn = LabelEncoder()
labels = labelEn.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
#veri array'e kaydedilmeden önce reshape edildiğinden array den okununca da düzgün gelir. yeniden reshape etmeye gerek yok
#eğer array'e atarken son değer 3 olarak yazılmasaydı burada reshape gerekirdi

#data =  data.reshape(-1,image_width , image_height , 3)




# train -test split
#%10 test %80 eğitim seti olacak şekilde böl
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = .10, shuffle = True)
#shuffle görüntüleri karıştırmaya yarıyor


print(
"""
x_train shape: {}
x_test shape: {}
y_train shape: {}
y_test shape: {}

""".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))

In [ ]:
#Standart sapmayı ve ortalamayı kullanarak normalizasyon işlemi yapılıyor.

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [ ]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = .10, shuffle = True,random_state=42)
# random_state rastgele seçilen değerlerin bir sonraki aşamada aynı değerler ile devam edilmesini sağlar.

In [ ]:
# Kayıp Fonkisyonunu en aza indirmek için optimizer kullanılıyor.
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
epc = 30 # öğrenme iterasyon sayısı genelde en az 100 kere olması gerekiyor
bs = 8

In [ ]:
# Keras kütüphanesinde hazır olan Xception modelini kullanarak eğitme işlemi yapılacak.
import tensorflow as tf

base_model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
NUM_CLASSES = len(classes)# kategorilerin sayısı alınıyor.


model = tf.keras.models.Sequential()
model.add(base_model)
model.add(Flatten()) # Matrisi bir vektöre dönüştürmek için Flatten işlemi uyguluyoruz.
model.add(Dropout(0.5)) # Overfitting aşırı uyumu önlemek için Dropout ekledik.
model.add(Dense(NUM_CLASSES, activation='softmax')) # sınıflandırma için sofmax fonksiyonunu kullandık.

model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Modelin öğrenme sırasında beli bir epoch saysında iyileştirme katetmezse devereye girer ve modelin baştan öğrenmesini başlatır.
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1,  factor=0.5, min_lr=0.00001)

In [ ]:
# Modeli eğitmeye başlıyoruz.
history = model.fit(x_train,y_train, batch_size=bs,
                          epochs = epc, validation_data = (x_validate,y_validate),
                          verbose = 1, callbacks=[learning_rate_reduction])

In [ ]:
# modeli test verisi üzerinde çalıştırıp ilk on tahminin doğruluğuna bakıyorum.
model.evaluate(x_test, y_test)
ypred = model.predict(x_test)
ypred1 = [np.argmax(element) for element in ypred]
print(ypred1[:10])
print(y_test[:10])

In [ ]:
# yapılan tahminlerin kaçı kaçı yanlış sayan fonkisyon.
def count_the_Wrong_pred(ypred, y_test):
  trues = 0
  wrongs = 0
  for i in range(len(ypred)):

    if y_test[i][int(ypred[i])] == 1.0:
      trues += 1
    else:
      wrongs += 1

  return trues, wrongs

In [ ]:
# Yanlış ve doğru yapılan tahminleri yazdırıyorum.
trues, wrongs = count_the_Wrong_pred(ypred1, y_test)
print("trures:" , trues)
print("wrongs:" , wrongs)

In [ ]:
# Doğruluk oranını graphla çizidiriyoruz.
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# yanlışlık oranların grafı.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()